# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
import pickle
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_data.db')
df = pd.read_sql('SELECT * FROM disaster_data', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# load data from database
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(w).lower().strip() for w in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))
                     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
print('****************************************Train Data Report****************************************')
y_pred_train = pipeline.predict(X_train)
print(classification_report(y_train.values, y_pred_train, target_names=Y.columns.values))
print('****************************************Text Data Report****************************************')
y_pred_test = pipeline.predict(X_test)
print(classification_report(y_test.values, y_pred_test, target_names=Y.columns.values))

****************************************Train Data Report****************************************
                        precision    recall  f1-score   support

               related       0.99      1.00      0.99     15097
               request       1.00      0.92      0.96      3318
                 offer       0.99      0.76      0.86        90
           aid_related       1.00      0.96      0.98      8126
          medical_help       1.00      0.84      0.91      1600
      medical_products       1.00      0.85      0.92       996
     search_and_rescue       1.00      0.80      0.89       558
              security       1.00      0.74      0.85       354
              military       1.00      0.85      0.92       628
                 water       1.00      0.92      0.96      1279
                  food       1.00      0.93      0.96      2177
               shelter       1.00      0.91      0.95      1736
              clothing       1.00      0.85      0.92       300
     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {
    #'vect__ngram_range': ((1, 1), (1, 2)),
    #'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),
    #'clf__estimator__n_estimators': [50, 100, 200],
    #'clf__estimator__min_samples_split': [2, 3, 4]
}

In [9]:
cv = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

In [10]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  5.4min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
print('****************************************Train Data Report****************************************')
y_pred_train = cv.predict(X_train)
print(classification_report(y_train.values, y_pred_train, target_names=Y.columns.values))
print('****************************************Text Data Report****************************************')
y_pred_test = cv.predict(X_test)
print(classification_report(y_test.values, y_pred_test, target_names=Y.columns.values))

****************************************Train Data Report****************************************
                        precision    recall  f1-score   support

               related       0.99      1.00      1.00     15097
               request       1.00      0.92      0.96      3318
                 offer       0.98      0.67      0.79        90
           aid_related       1.00      0.97      0.98      8126
          medical_help       1.00      0.87      0.93      1600
      medical_products       1.00      0.85      0.92       996
     search_and_rescue       1.00      0.76      0.86       558
              security       1.00      0.75      0.85       354
              military       1.00      0.87      0.93       628
                 water       1.00      0.90      0.95      1279
                  food       1.00      0.94      0.97      2177
               shelter       1.00      0.91      0.95      1736
              clothing       1.00      0.82      0.90       300
     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
cv.best_params_

{'tfidf__use_idf': True}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier

In [14]:
pipeline_ada = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(AdaBoostClassifier()))
                     ])

In [15]:
pipeline_ada.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [16]:
print('****************************************Train Data Report****************************************')
y_pred_train = pipeline_ada.predict(X_train)
print(classification_report(y_train.values, y_pred_train, target_names=Y.columns.values))
print('****************************************Text Data Report****************************************')
y_pred_test = pipeline_ada.predict(X_test)
print(classification_report(y_test.values, y_pred_test, target_names=Y.columns.values))

****************************************Train Data Report****************************************
                        precision    recall  f1-score   support

               related       0.83      0.95      0.88     15097
               request       0.79      0.54      0.64      3318
                 offer       0.28      0.06      0.09        90
           aid_related       0.77      0.61      0.68      8126
          medical_help       0.67      0.27      0.39      1600
      medical_products       0.73      0.36      0.48       996
     search_and_rescue       0.68      0.22      0.33       558
              security       0.43      0.06      0.11       354
              military       0.66      0.36      0.46       628
                 water       0.78      0.65      0.71      1279
                  food       0.82      0.69      0.75      2177
               shelter       0.81      0.53      0.64      1736
              clothing       0.81      0.53      0.64       300
     

In [17]:
pipeline_grad = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(GradientBoostingClassifier()))
                     ])

pipeline_grad.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
           n_jobs=1))])

In [18]:
print('****************************************Train Data Report****************************************')
print(classification_report(y_train.values, y_pred_train, target_names=Y.columns.values))
print('****************************************Text Data Report****************************************')
print(classification_report(y_test.values, y_pred_test, target_names=Y.columns.values))

****************************************Train Data Report****************************************
                        precision    recall  f1-score   support

               related       0.83      0.95      0.88     15097
               request       0.79      0.54      0.64      3318
                 offer       0.28      0.06      0.09        90
           aid_related       0.77      0.61      0.68      8126
          medical_help       0.67      0.27      0.39      1600
      medical_products       0.73      0.36      0.48       996
     search_and_rescue       0.68      0.22      0.33       558
              security       0.43      0.06      0.11       354
              military       0.66      0.36      0.46       628
                 water       0.78      0.65      0.71      1279
                  food       0.82      0.69      0.75      2177
               shelter       0.81      0.53      0.64      1736
              clothing       0.81      0.53      0.64       300
     

### 9. Export your model as a pickle file

In [20]:
filename = 'classifier.sav'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.